In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
df1= pd.read_csv('/content/case_study1.xlsx - case_study1.csv')
df2= pd.read_csv('/content/case_study2.xlsx - case_study2.csv')

In [ ]:
df1.head(3)

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL
0,1,5,4,1,0,0,0.000,0.0,0.2,0.8,...,0,0,1,0,4,1,4,0,72,18
1,2,1,0,1,0,0,0.000,0.0,1.0,0.0,...,0,1,0,0,0,0,1,0,7,7
2,3,8,0,8,1,0,0.125,0.0,1.0,0.0,...,0,6,1,0,0,2,6,0,47,2


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51336 entries, 0 to 51335
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   PROSPECTID            51336 non-null  int64  
 1   Total_TL              51336 non-null  int64  
 2   Tot_Closed_TL         51336 non-null  int64  
 3   Tot_Active_TL         51336 non-null  int64  
 4   Total_TL_opened_L6M   51336 non-null  int64  
 5   Tot_TL_closed_L6M     51336 non-null  int64  
 6   pct_tl_open_L6M       51336 non-null  float64
 7   pct_tl_closed_L6M     51336 non-null  float64
 8   pct_active_tl         51336 non-null  float64
 9   pct_closed_tl         51336 non-null  float64
 10  Total_TL_opened_L12M  51336 non-null  int64  
 11  Tot_TL_closed_L12M    51336 non-null  int64  
 12  pct_tl_open_L12M      51336 non-null  float64
 13  pct_tl_closed_L12M    51336 non-null  float64
 14  Tot_Missed_Pmnt       51336 non-null  int64  
 15  Auto_TL            

In [ ]:
#null values in df1
for i in df1.columns:
    print(i,'---->',df1.loc[df1[i]==-99999].shape[0])

PROSPECTID ----> 0
Total_TL ----> 0
Tot_Closed_TL ----> 0
Tot_Active_TL ----> 0
Total_TL_opened_L6M ----> 0
Tot_TL_closed_L6M ----> 0
pct_tl_open_L6M ----> 0
pct_tl_closed_L6M ----> 0
pct_active_tl ----> 0
pct_closed_tl ----> 0
Total_TL_opened_L12M ----> 0
Tot_TL_closed_L12M ----> 0
pct_tl_open_L12M ----> 0
pct_tl_closed_L12M ----> 0
Tot_Missed_Pmnt ----> 0
Auto_TL ----> 0
CC_TL ----> 0
Consumer_TL ----> 0
Gold_TL ----> 0
Home_TL ----> 0
PL_TL ----> 0
Secured_TL ----> 0
Unsecured_TL ----> 0
Other_TL ----> 0
Age_Oldest_TL ----> 40
Age_Newest_TL ----> 40


In [ ]:
df1.loc[df1['Age_Oldest_TL']==-99999]
#Rows with missing values in the Age_Oldest_TL column also have missing values in the Age_Newest_TL column
df1=df1.loc[df1['Age_Oldest_TL']!=-99999]
#for i in df1.columns:
#   print(df1.loc[df1[i]==-99999].shape, i)



In [ ]:
df1.duplicated().sum()

0

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51336 entries, 0 to 51335
Data columns (total 62 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   PROSPECTID                    51336 non-null  int64  
 1   time_since_recent_payment     51336 non-null  int64  
 2   time_since_first_deliquency   51336 non-null  int64  
 3   time_since_recent_deliquency  51336 non-null  int64  
 4   num_times_delinquent          51336 non-null  int64  
 5   max_delinquency_level         51336 non-null  int64  
 6   max_recent_level_of_deliq     51336 non-null  int64  
 7   num_deliq_6mts                51336 non-null  int64  
 8   num_deliq_12mts               51336 non-null  int64  
 9   num_deliq_6_12mts             51336 non-null  int64  
 10  max_deliq_6mts                51336 non-null  int64  
 11  max_deliq_12mts               51336 non-null  int64  
 12  num_times_30p_dpd             51336 non-null  int64  
 13  n

In [ ]:
droping_col=[]
for i in df2.columns:
    print(df2.loc[df2[i]==-99999].shape[0],'---->',i)
    if df2.loc[df2[i]==-99999].shape[0]>10000:           #that is approx 20% of the data
        droping_col.append(i)
print(droping_col)
print('length',len(droping_col))


#dropping the columns
df2=df2.drop(columns=droping_col)
print(df2.shape)

for i in df2.columns:
    df2=df2.loc[df2[i]!=-99999]

0 ----> PROSPECTID
4291 ----> time_since_recent_payment
35949 ----> time_since_first_deliquency
35949 ----> time_since_recent_deliquency
0 ----> num_times_delinquent
35949 ----> max_delinquency_level
0 ----> max_recent_level_of_deliq
0 ----> num_deliq_6mts
0 ----> num_deliq_12mts
0 ----> num_deliq_6_12mts
12890 ----> max_deliq_6mts
10832 ----> max_deliq_12mts
0 ----> num_times_30p_dpd
0 ----> num_times_60p_dpd
0 ----> num_std
0 ----> num_std_6mts
0 ----> num_std_12mts
0 ----> num_sub
0 ----> num_sub_6mts
0 ----> num_sub_12mts
0 ----> num_dbt
0 ----> num_dbt_6mts
0 ----> num_dbt_12mts
0 ----> num_lss
0 ----> num_lss_6mts
0 ----> num_lss_12mts
0 ----> recent_level_of_deliq
6321 ----> tot_enq
6321 ----> CC_enq
6321 ----> CC_enq_L6m
6321 ----> CC_enq_L12m
6321 ----> PL_enq
6321 ----> PL_enq_L6m
6321 ----> PL_enq_L12m
6321 ----> time_since_recent_enq
6321 ----> enq_L12m
6321 ----> enq_L6m
6321 ----> enq_L3m
0 ----> MARITALSTATUS
0 ----> EDUCATION
0 ----> AGE
0 ----> GENDER
0 ----> NETMONTHL

In [ ]:
df2.shape

(42066, 54)

In [ ]:
#checking the common column in both dataset
for i in df1.columns:
    if i in df2.columns:
        print(i)



PROSPECTID


In [ ]:
df2['PROSPECTID']
df1.PROSPECTID


0            1
1            2
2            3
3            4
4            5
         ...  
51331    51332
51332    51333
51333    51334
51334    51335
51335    51336
Name: PROSPECTID, Length: 51296, dtype: int64

In [ ]:
df= pd.merge(df1,df2,how='inner',on=['PROSPECTID'])  #common chize hi aayenge
df.head(3)

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,5,4,1,0,0,0.000,0.0,0.2,0.8,...,0.0,0.0,0.0,0.0,1,0,PL,PL,696,P2
1,2,1,0,1,0,0,0.000,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,8,0,8,1,0,0.125,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1,0,ConsumerLoan,others,693,P2


In [ ]:
#df.info()
categorical_col=[]
for i in df.columns:
    if df[i].dtype=='object':

       categorical_col.append(i)
categorical_col= categorical_col[:-1]      # last column was output column

categorical_col

['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']

In [ ]:
# chi square test

for i in categorical_col:
    chi2 , pval,_,_= chi2_contingency(pd.crosstab(df[i], df['Approved_Flag']))
    print(i,'--->',pval)

# p value of all categorical feature is less than 0.05, we will accept all feature

MARITALSTATUS ---> 3.578180861038862e-233
EDUCATION ---> 2.6942265249737532e-30
GENDER ---> 1.907936100186563e-05
last_prod_enq2 ---> 0.0
first_prod_enq2 ---> 7.84997610555419e-287


In [ ]:
#numerical columns

num_col= []
for i in df.columns:
    if df[i].dtype != 'object':
        num_col.append(i)
num_col=num_col[1:]
print(len(num_col))
num_col

72


['Total_TL',
 'Tot_Closed_TL',
 'Tot_Active_TL',
 'Total_TL_opened_L6M',
 'Tot_TL_closed_L6M',
 'pct_tl_open_L6M',
 'pct_tl_closed_L6M',
 'pct_active_tl',
 'pct_closed_tl',
 'Total_TL_opened_L12M',
 'Tot_TL_closed_L12M',
 'pct_tl_open_L12M',
 'pct_tl_closed_L12M',
 'Tot_Missed_Pmnt',
 'Auto_TL',
 'CC_TL',
 'Consumer_TL',
 'Gold_TL',
 'Home_TL',
 'PL_TL',
 'Secured_TL',
 'Unsecured_TL',
 'Other_TL',
 'Age_Oldest_TL',
 'Age_Newest_TL',
 'time_since_recent_payment',
 'num_times_delinquent',
 'max_recent_level_of_deliq',
 'num_deliq_6mts',
 'num_deliq_12mts',
 'num_deliq_6_12mts',
 'num_times_30p_dpd',
 'num_times_60p_dpd',
 'num_std',
 'num_std_6mts',
 'num_std_12mts',
 'num_sub',
 'num_sub_6mts',
 'num_sub_12mts',
 'num_dbt',
 'num_dbt_6mts',
 'num_dbt_12mts',
 'num_lss',
 'num_lss_6mts',
 'num_lss_12mts',
 'recent_level_of_deliq',
 'tot_enq',
 'CC_enq',
 'CC_enq_L6m',
 'CC_enq_L12m',
 'PL_enq',
 'PL_enq_L6m',
 'PL_enq_L12m',
 'time_since_recent_enq',
 'enq_L12m',
 'enq_L6m',
 'enq_L3m',

In [ ]:
vif_data= df[num_col]
total_column= vif_data.shape[1]
column_index=0
column_to_be_kept=[]

for i in range(total_column):
    vif_val= variance_inflation_factor(vif_data,column_index)
    print(num_col[column_index],'--->',vif_val)

    if vif_val < 6:
        column_to_be_kept.append(num_col[i])
        column_index=column_index+1
    else:
        vif_data= vif_data.drop(columns=num_col[i])






/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Total_TL ---> inf


/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Total_TL ---> inf
Total_TL ---> 11.320180023967996
Total_TL ---> 8.363698035000336
Total_TL ---> 6.520647877790928
Total_TL ---> 5.149501618212625
Tot_Closed_TL ---> 2.611111040579735


/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Tot_Active_TL ---> inf
Tot_Active_TL ---> 1788.7926256209232
Tot_Active_TL ---> 8.601028256477228
Tot_Active_TL ---> 3.832800792153077
Total_TL_opened_L6M ---> 6.099653381646723
Total_TL_opened_L6M ---> 5.581352009642766
Tot_TL_closed_L6M ---> 1.985584353098778


/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


pct_tl_open_L6M ---> inf
pct_tl_open_L6M ---> 4.80953830281934
pct_tl_closed_L6M ---> 23.270628983464636
pct_tl_closed_L6M ---> 30.595522588100053
pct_tl_closed_L6M ---> 4.384346405965583
pct_active_tl ---> 3.0646584155234238
pct_closed_tl ---> 2.898639771299251
Total_TL_opened_L12M ---> 4.377876915347324
Tot_TL_closed_L12M ---> 2.207853583695844
pct_tl_open_L12M ---> 4.916914200506864
pct_tl_closed_L12M ---> 5.214702030064725
Tot_Missed_Pmnt ---> 3.3861625024231476
Auto_TL ---> 7.840583309478997
Auto_TL ---> 5.255034641721434


/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


CC_TL ---> inf
CC_TL ---> 7.380634506427238
CC_TL ---> 1.4210050015175733
Consumer_TL ---> 8.083255010190316
Consumer_TL ---> 1.6241227524040114
Gold_TL ---> 7.257811920140003
Gold_TL ---> 15.59624383268298
Gold_TL ---> 1.825857047132431
Home_TL ---> 1.5080839450032664
PL_TL ---> 2.172088834824578
Secured_TL ---> 2.6233975535272274
Unsecured_TL ---> 2.2959970812106176
Other_TL ---> 7.360578319196446
Other_TL ---> 2.1602387773102567
Age_Oldest_TL ---> 2.8686288267891467
Age_Newest_TL ---> 6.458218003637272
Age_Newest_TL ---> 2.8474118865638247
time_since_recent_payment ---> 4.753198156284083
num_times_delinquent ---> 16.22735475594825
num_times_delinquent ---> 6.424377256363877
num_times_delinquent ---> 8.887080381808678
num_times_delinquent ---> 2.3804746142952653
max_recent_level_of_deliq ---> 8.60951347651454
max_recent_level_of_deliq ---> 13.06755093547673
max_recent_level_of_deliq ---> 3.500040056654653
num_deliq_6mts ---> 1.9087955874813773
num_deliq_12mts ---> 17.006562234161628


In [ ]:
len(column_to_be_kept)
    #After checking the VIF values, only 39 columns have a VIF greater than 6
#column_to_be_kept

39

In [ ]:
from scipy.stats import f_oneway

columns_to_be_kept_numerical = []

for i in column_to_be_kept:
    a = list(df[i])
    b = list(df['Approved_Flag'])

    group_P1 = [value for value, group in zip(a, b) if group == 'P1']
    group_P2 = [value for value, group in zip(a, b) if group == 'P2']
    group_P3 = [value for value, group in zip(a, b) if group == 'P3']
    group_P4 = [value for value, group in zip(a, b) if group == 'P4']


    f_statistic, p_value = f_oneway(group_P1, group_P2, group_P3, group_P4)

    if p_value <= 0.05:
        columns_to_be_kept_numerical.append(i)







In [ ]:
len(columns_to_be_kept_numerical)

37

In [ ]:
df['Approved_Flag'].value_counts()

Approved_Flag
P2    25452
P3     6440
P4     5264
P1     4908
Name: count, dtype: int64

# feature engineering


In [ ]:
categorical_col

['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']

In [ ]:
for i in categorical_col:
    print(i,'---->',df[i].unique())

MARITALSTATUS ----> ['Married' 'Single']
EDUCATION ----> ['12TH' 'GRADUATE' 'SSC' 'POST-GRADUATE' 'UNDER GRADUATE' 'OTHERS'
 'PROFESSIONAL']
GENDER ----> ['M' 'F']
last_prod_enq2 ----> ['PL' 'ConsumerLoan' 'AL' 'CC' 'others' 'HL']
first_prod_enq2 ----> ['PL' 'ConsumerLoan' 'others' 'AL' 'HL' 'CC']


In [ ]:
features = columns_to_be_kept_numerical + ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']
df = df[features + ['Approved_Flag']]


In [ ]:
df['EDUCATION']=df['EDUCATION'].replace({
    'SSC': 1,
    '12TH':2,
    'GRADUATE':3,
    'UNDER GRADUATE':3,
    'POST-GRADUATE':4,
    'OTHERS':1,
    'PROFESSIONAL':4





})

In [ ]:
df_encoded=pd.get_dummies(df,columns=['MARITALSTATUS','GENDER','last_prod_enq2','first_prod_enq2'],dtype='int')
df_encoded


,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,...,last_prod_enq2_ConsumerLoan,last_prod_enq2_HL,last_prod_enq2_PL,last_prod_enq2_others,first_prod_enq2_AL,first_prod_enq2_CC,first_prod_enq2_ConsumerLoan,first_prod_enq2_HL,first_prod_enq2_PL,first_prod_enq2_others
0,0.000,0.00,0,0.000,0,0,0,4,1,4,...,0,0,1,0,0,0,0,0,1,0
1,0.000,0.00,0,0.000,0,0,0,0,0,1,...,1,0,0,0,0,0,1,0,0,0
2,0.125,0.00,0,0.000,1,0,0,0,2,6,...,1,0,0,0,0,0,0,0,0,1
3,0.000,0.00,0,0.000,0,0,0,0,3,0,...,0,0,0,0,1,0,0,0,0,0
4,0.000,0.00,1,0.167,0,0,0,0,6,0,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42059,0.333,0.00,0,0.000,0,0,0,0,0,3,...,1,0,0,0,0,0,1,0,0,0
42060,0.000,0.25,1,0.250,0,0,0,0,2,2,...,0,0,0,1,0,0,0,0,0,1
42061,0.500,0.50,1,0.500,0,0,0,0,0,2,...,1,0,0,0,0,0,0,0,0,1
42062,0.000,0.00,1,0.500,0,0,0,0,0,2,...,1,0,0,0,0,0,0,0,0,1


In [ ]:
df['EDUCATION'].value_counts()

EDUCATION
3    18712
2    11703
1     9532
4     2117
Name: count, dtype: int64

In [ ]:
df_encoded['EDUCATION']

0        2
1        3
2        1
3        4
4        2
        ..
42059    2
42060    1
42061    1
42062    3
42063    1
Name: EDUCATION, Length: 42064, dtype: int64

In [ ]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 55 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   pct_tl_open_L6M               42064 non-null  float64
 1   pct_tl_closed_L6M             42064 non-null  float64
 2   Tot_TL_closed_L12M            42064 non-null  int64  
 3   pct_tl_closed_L12M            42064 non-null  float64
 4   Tot_Missed_Pmnt               42064 non-null  int64  
 5   CC_TL                         42064 non-null  int64  
 6   Home_TL                       42064 non-null  int64  
 7   PL_TL                         42064 non-null  int64  
 8   Secured_TL                    42064 non-null  int64  
 9   Unsecured_TL                  42064 non-null  int64  
 10  Other_TL                      42064 non-null  int64  
 11  Age_Oldest_TL                 42064 non-null  int64  
 12  Age_Newest_TL                 42064 non-null  int64  
 13  t

In [ ]:
x= df_encoded.drop(columns='Approved_Flag')
y=df_encoded['Approved_Flag']

In [ ]:
y
x
x_train, x_test,y_train,y_test= train_test_split(x,y, test_size=0.2,random_state=42)

In [ ]:
rf= RandomForestClassifier()
rf.fit(x_train,y_train)
y_pred=rf.predict(x_test)
accuracy=accuracy_score(y_test, y_pred)
accuracy

0.7632235825508142

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)


for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()


Class p1:
Precision: 0.831002331002331
Recall: 0.703155818540434
F1 Score: 0.7617521367521368

Class p2:
Precision: 0.7955819881053526
Recall: 0.9280475718533201
F1 Score: 0.8567246111619398

Class p3:
Precision: 0.4401840490797546
Recall: 0.21660377358490565
F1 Score: 0.29033889731917045

Class p4:
Precision: 0.7259332023575639
Recall: 0.718172983479106
F1 Score: 0.7220322423058134



In [ ]:
#xgboost
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

xgb_c = xgb.XGBClassifier(objective='multi:softmax',  num_class=4)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)




xgb_c.fit(x_train,y_train)
yp1=xgb_c.predict(x_test)
accuracy_score(y_test,yp1)

0.7743967669083561

In [ ]:

precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, yp1)


for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()


Class p1:
Precision: 0.828819068255688
Recall: 0.7544378698224852
F1 Score: 0.7898812596799174

Class p2:
Precision: 0.8233082706766918
Recall: 0.911595639246779
F1 Score: 0.865205530994262

Class p3:
Precision: 0.4554794520547945
Recall: 0.3011320754716981
F1 Score: 0.3625624716038165

Class p4:
Precision: 0.7315175097276264
Recall: 0.7308066083576288
F1 Score: 0.7311618862421001



In [ ]:
from sklearn.model_selection import  GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
}

grid_search = GridSearchCV(estimator=xgb_c, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate the model with the best hyperparameters on the test set
best_model = grid_search.best_estimator_
accuracy = accuracy_score( best_model.predict(x_test),y_test)
print("Test Accuracy:", accuracy)


Best Hyperparameters: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 200}
Test Accuracy: 0.7815285867110424


In [ ]:
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test,best_model.predict(x_test) )


for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()

Class p1:
Precision: 0.8475877192982456
Recall: 0.76232741617357
F1 Score: 0.8026998961578401

Class p2:
Precision: 0.8170264623955432
Recall: 0.9302279484638256
F1 Score: 0.8699601445917139

Class p3:
Precision: 0.47831978319783197
Recall: 0.2664150943396226
F1 Score: 0.3422200678623364

Class p4:
Precision: 0.7419038272816487
Recall: 0.7346938775510204
F1 Score: 0.7382812500000001

